<a href="https://colab.research.google.com/github/nntadotzip/vi-augmentation/blob/main/BertPairClassification_SynonymReplacementMethod_2610sem0_8627sem1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import and Install

**Models**

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install datasets==1.0.1 
!pip install transformers==3.1.0

     |████████████████████████████████| 1.8 MB 7.9 MB/s 
     |████████████████████████████████| 212 kB 62.1 MB/s 
     |████████████████████████████████| 884 kB 8.4 MB/s 
     |████████████████████████████████| 1.2 MB 52.3 MB/s 
     |████████████████████████████████| 895 kB 56.2 MB/s 
     |████████████████████████████████| 3.0 MB 48.6 MB/s 


In [3]:
import torch
import torch.nn as nn
import os
import matplotlib.pyplot as plt
import copy
import torch.optim as optim
import random
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import autocast, GradScaler
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel, AdamW, get_linear_schedule_with_warmup
from datasets import load_dataset, load_metric

os.environ["TOKENIZERS_PARALLELISM"] = "false"

PyTorch version 1.10.0+cu111 available.
TensorFlow version 2.8.0 available.


In [4]:
# Check that we are using 100% of GPU memory footprint support libraries/code
# from https://github.com/patrickvonplaten/notebooks/blob/master/PyTorch_Reformer.ipynb
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip -q install gputil
!pip -q install psutil
!pip -q install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 11.9 GB  | Proc size: 1.6 GB
GPU RAM Free: 15109MB | Used: 0MB | Util   0% | Total 15109MB


**Cosine Similarity**

In [5]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [6]:
import string
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

# Load the Dataset

In [ ]:
# from google.colab import files
# files.upload()

In [7]:
augmented_sem0_1 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/Augmentation/backtranslation_results/backtrans_1to1999_sem0.csv', encoding='utf8')
augmented_sem0_2 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/Augmentation/backtranslation_results/backtrans_1to1999_sem0_SWAPPED.csv', encoding='utf8')

In [8]:
augmented_sem0_1 = augmented_sem0_1[['q1_en_to_vi', 'q2_fr_to_vi']]
augmented_sem0_1['ans_question_1'] = ''
augmented_sem0_1['ans_question_2'] = ''
augmented_sem0_1['cosine_similarity'] = ''
augmented_sem0_1['manual_label'] = 0
augmented_sem0_1  = augmented_sem0_1[['q1_en_to_vi', 'q2_fr_to_vi', 'ans_question_1', 'ans_question_2', 'cosine_similarity','manual_label']]
augmented_sem0_1 = augmented_sem0_1.set_axis(['question_1', 'question_2', 'ans_question_1', 'ans_question_2', 'cosine_similarity','manual_label'], axis=1, inplace=False)

augmented_sem0_2 = augmented_sem0_2[['q1_en_to_vi', 'q2_fr_to_vi']]
augmented_sem0_2['ans_question_1'] = ''
augmented_sem0_2['ans_question_2'] = ''
augmented_sem0_2['cosine_similarity'] = ''
augmented_sem0_2['manual_label'] = 0
augmented_sem0_2  = augmented_sem0_2[['q1_en_to_vi', 'q2_fr_to_vi', 'ans_question_1', 'ans_question_2', 'cosine_similarity','manual_label']]
augmented_sem0_2 = augmented_sem0_2.set_axis(['question_1', 'question_2', 'ans_question_1', 'ans_question_2', 'cosine_similarity','manual_label'], axis=1, inplace=False)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [9]:
augmented_sem0 = augmented_sem0_1.append(augmented_sem0_2, sort=False, ignore_index=True)
augmented_sem0

,question_1,question_2,ans_question_1,ans_question_2,cosine_similarity,manual_label
0,"Dân số và lao động |Dân số Đông, nguồn lao động dồi dào và có trình độ.|.Lực lượng lao động lớn nhất trong cả nước.|.Công nhân có trình độ cao nhất trên toàn quốc.|.Lao động tập trung chủ yếu các thành phố.","Các biện pháp cơ bản để mang lại đồng bằng sông Hồng sẽ sớm trở thành khu vực sản xuất thực phẩm, bất động sản là: chú ý đến các sản phẩm và thị trường.|.Thay đổi cấu trúc của cây trồng và cấu trúc theo mùa.|.Hãy chú ý đến môi trường và bảo vệ tài nguyên đất.|.Mạnh mẽ phát triển văn hóa trồng trọt.",,,,0
1,"Dân số và lao động |Dân số Đông, nguồn lao động dồi dào và có trình độ.|.Lực lượng lao động lớn nhất trong cả nước.|.Công nhân có trình độ cao nhất trên toàn quốc.|.Lao động tập trung chủ yếu các thành phố.","Hướng về định hướng tái cấu trúc trong lĩnh vực kinh tế nội bộ ở Delta Red River là: |Phát triển và hiện đại hóa nông nghiệp, gắn phát triển với ngành công nghiệp chế biến.|.Phát triển và kịp thời của các nhà khai thác công nghiệp chuyển đổi, trong khi các ngành công nghiệp và dịch vụ khác có liên quan đến điều kiện nông nghiệp của hàng hóa.|.Phát triển và hiện đại hóa khai thác công nghiệp, gắn nó với các sản phẩm nông nghiệp.|.Phát triển và hiện đại hóa ngành chế biến và khai thác.",,,,0
2,"Dân số và lao động |Dân số Đông, nguồn lao động dồi dào và có trình độ.|.Lực lượng lao động lớn nhất trong cả nước.|.Công nhân có trình độ cao nhất trên toàn quốc.|.Lao động tập trung chủ yếu các thành phố.","Câu hỏi kinh tế xã hội là mối quan tâm lớn ở đồng bằng sông Hồng trong giai đoạn hiện tại: |Các lĩnh vực chính của sản xuất thực phẩm và sản xuất thực phẩm.|.Dân số Đông, diện tích đất canh tác hạn chế.|.Trình độ chuyên môn với cường độ cao.|.Nơi tập trung vào nhiều trung tâm kinh tế, văn hóa và chính trị lớn trên toàn quốc.",,,,0
3,"Dân số và lao động |Dân số Đông, nguồn lao động dồi dào và có trình độ.|.Lực lượng lao động lớn nhất trong cả nước.|.Công nhân có trình độ cao nhất trên toàn quốc.|.Lao động tập trung chủ yếu các thành phố.","Dựa trên Địa lý Atlat Việt Nam, vui lòng cho biết các tỉnh nào sau đây của đồng bằng sông Hồng không liền kề với biển?|.Hưng Yên, Hải Phòng.|.Hà Nam, Bắc Ninh.|.Hà Nam, Ninh Bình.|.Đinh Man, Bắc Ninh.",,,,0
4,"Dân số và lao động |Dân số Đông, nguồn lao động dồi dào và có trình độ.|.Lực lượng lao động lớn nhất trong cả nước.|.Công nhân có trình độ cao nhất trên toàn quốc.|.Lao động tập trung chủ yếu các thành phố.","Dựa trên Atlat địa lý của Việt Nam, vui lòng cho biết trung tâm công nghiệp nào trong đồng bằng sông Hồng có giá trị sản xuất công nghiệp từ 40 đến 120 nghìn tỷ đồng không?|.Hà Nội.|.Hải Phòng.|.Phúc Yên.|.Ninh Bac.|.Ninh Bac.|.",,,,0
...,...,...,...,...,...,...
2605,"Lý do cơ bản cho đồng bằng sông Hồng có mật độ dân số cao hơn đồng bằng sông Cửu Long là |Đất màu mỡ, màu mỡ hơn.|.Khí hậu thuận tiện hơn.|.Giao thông thuận tiện hơn. |Lịch sử giải quyết sớm hơn.","Sự phát triển kinh tế xã hội của các thành viên của dân tộc thiểu số của đất nước chúng ta nên tập trung vào ""các nhóm dân tộc đóng vai trò rất quan trọng trong an ninh quốc gia.|.Một số dân tộc thiểu số có kinh nghiệm sản xuất có giá trị.|.Phát triển kinh tế xã hội của các dân tộc có sự khác biệt đáng kể, mức sống của Sở Dân tộc thiểu số.|.Chúng tôi đã không tập trung trước đây trước vấn đề này.",,,,0
2606,Nguyên nhân cơ bản nhất của Delta Red River có mật độ dân số cao nhất trên toàn quốc là lịch sử khai thác lãnh thổ lâu đời nhất.|.Khí hậu thuận lợi và nhẹ.|.Có những nền kinh tế rất phát triển.|.Có lực lượng sản xuất có trình độ cao và chuyên nghiệp.,"Sự phát triển kinh tế xã hội của các thành viên của dân tộc thiểu số của đất nước chúng ta nên tập trung vào ""các nhóm dân tộc đóng vai trò rất quan trọng trong an ninh quốc gia.|.Một số dân tộc thiểu số có kinh nghiệm sản xuất có giá trị.|.Phát triển kinh tế xã hội của các dân tộc có sự khác biệt đáng kể, mức sống của Sở Dân tộc thiểu số.|.Chúng tôi đã không tập trung trước đây t

In [10]:
original_df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/Augmentation/prep/vietjackGeo/labeled_1to1999/labeled_1to2000_csvformat.csv', encoding='utf8')
original_df.head(2)

,question_1,question_2,ans_question_1,ans_question_2,cosine_similarity,manual_label
0,"“dân cư và lao động”|dân số đông, nguồn lao động dồi dào và có trình độ.|nguồn lao động lớn nhất cả nước.|lao động có trình độ cao nhất cả nước.|lao động tập trung chủ yếu ở các thành phố lớn.","Biện pháp cơ bản để đưa đồng bằng sông Hồng sớm trở thành vùng sản xuất lương thực, thực phầm hàng hóa là:|quan tâm đến chất lương sản phẩm và thị trường.|thay đổi cơ cấu cây cây trồng và cơ cấu mùa vụ.|chú ý đến môi trường và bảo vệ tài nguyên đất.|phát triển mạnh cây vụ đông.","dân số đông, nguồn lao động dồi dào và có trình độ.",thay đổi cơ cấu cây cây trồng và cơ cấu mùa vụ.,"0,71565396",0.0
1,"“dân cư và lao động”|dân số đông, nguồn lao động dồi dào và có trình độ.|nguồn lao động lớn nhất cả nước.|lao động có trình độ cao nhất cả nước.|lao động tập trung chủ yếu ở các thành phố lớn.","Tại sao việc làm là một trong những vấn đề nan giải ở Đồng bằng sông Hồng nhất là ở khu vực thành thị?|Do dân nhập cư đông.|Do dân số đông, kết cấu dân số trẻ.|Do nền kinh tế còn chậm phát triển.|Do dân số đông, kết cấu dân số trẻ trong điều kiện kinh tế chậm","dân số đông, nguồn lao động dồi dào và có trình độ.","Do dân số đông, kết cấu dân số trẻ trong điều kiện kinh tế chậm","0,8153097",1.0


In [11]:
augmented_df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/Augmentation/prep/vietjackGeo/final_augment/syn_augmented_1to1999.csv', encoding='utf8')
augmented_df

,sentence1,sentence2,ans_question_1,ans_question_2,manual_label,augmented1,augmented2
0,"“dân cư và lao động”|dân số đông, nguồn lao động dồi dào và có trình độ.|nguồn lao động lớn nhất cả nước.|lao động có trình độ cao nhất cả nước.|lao động tập trung chủ yếu ở các thành phố lớn.","Tại sao việc làm là một trong những vấn đề nan giải ở Đồng bằng sông Hồng nhất là ở khu vực thành thị?|Do dân nhập cư đông.|Do dân số đông, kết cấu dân số trẻ.|Do nền kinh tế còn chậm phát triển.|Do dân số đông, kết cấu dân số trẻ trong điều kiện kinh tế chậm","dân số đông, nguồn lao động dồi dào và có trình độ.","Do dân số đông, kết cấu dân số trẻ trong điều kiện kinh tế chậm",1.0,"“ dân cư và lao động ” | dân cư đông , nguồn lao động dồi dào và có trình độ .| nguồn lao động lớn nhất cả nước .| lao động có trình độ cao nhất cả nước .| lao động tập trung chủ yếu ở các thành phố lớn .","tại sao việc làm là một trong những vấn đề nan giải ở đồng bằng sông hồng nhất là ở khu vực thành thị ?| do dân nhập cư đông .| do dân cư đông , kết cấu dân cư trẻ .| do nền kinh tế còn chậm phát triển .| do dân cư đông , kết cấu dân cư trẻ trong điều kiện kinh tế chậm"
1,"Bão, lũ lụt, hạn hán, gió tây khô nóng là thiên tai xảy ra chủ yếu ở vùng|Đồng bằng sông Hồng.|Tây Bắc.|Duyên hải miền Trung.|Tây Nguyên",Vùng nào ở nước ta chịu ảnh hưởng mạnh mẽ nhất của gió Tây khô nóng?|Bắc Trung Bộ.|Đông Bắc.|Đông Nam Bộ.|Tây Nguyên.,Duyên hải miền Trung.,Bắc Trung Bộ.,1.0,"bão , lũ lụt , hạn hán , gió mạnh tây khô nóng là thiên tai xảy ra chủ yếu ở vùng | đồng bằng sông hồng .| tây bắc .| duyên hải miền trung .| tây nguyên",vùng nào ở nước ta chịu ảnh hưởng mạnh mẽ nhất của gió mạnh tây khô nóng ?| bắc trung bộ .| đông bắc .| đông nam bộ .| tây nguyên .
2,"Bão, lũ lụt, hạn hán, gió tây khô nóng là thiên tai xảy ra chủ yếu ở vùng|Đồng bằng sông Hồng.|Tây Bắc.|Duyên hải miền Trung.|Tây Nguyên",Vùng nào ở nước ta chịu ảnh hưởng mạnh mẽ nhất của gió Tây khô nóng?|Bắc Trung Bộ.|Đông Bắc.|Đông Nam Bộ.|Tây Nguyên.,Duyên hải miền Trung.,Bắc Trung Bộ.,1.0,"bão , lũ lụt , hạn hán , gió mạnh tây khô nóng là thiên tai xảy ra chủ yếu ở vùng | đồng bằng sông hồng .| tây bắc .| duyên hải miền trung .| tây nguyên",vùng ngoại ô nào ở nước ta chịu ảnh hưởng mạnh mẽ nhất của gió tây khô nóng ?| bắc trung bộ .| đông bắc .| đông nam bộ .| tây nguyên .
3,"Bão, lũ lụt, hạn hán, gió tây khô nóng là thiên tai xảy ra chủ yếu ở vùng|Đồng bằng sông Hồng.|Tây Bắc.|Duyên hải miền Trung.|Tây Nguyên",Vùng nào ở nước ta chịu ảnh hưởng mạnh mẽ nhất của gió Tây khô nóng?|Bắc Trung Bộ.|Đông Bắc.|Đông Nam Bộ.|Tây Nguyên.,Duyên hải miền Trung.,Bắc Trung Bộ.,1.0,"bão , lũ lụt , hạn hán , gió mạnh tây khô nóng là thiên tai xảy ra chủ yếu ở vùng | đồng bằng sông hồng .| tây bắc .| duyên hải miền trung .| tây nguyên",vùng nào ở nước ta chịu ảnh hưởng mạnh mẽ nhất của gió tây khô nóng ?| bắc trung bộ .| đông bắc .| đông nam bộ .| cao nguyên .
4,"Bão, lũ lụt, hạn hán, gió tây khô nóng là thiên tai xảy ra chủ yếu ở vùng|Đồng bằng sông Hồng.|Tây Bắc.|Duyên hải miền Trung.|Tây Nguyên",Vùng nào ở nước ta chịu ảnh hưởng mạnh mẽ nhất của gió Tây khô nóng?|Bắc Trung Bộ.|Đông Bắc.|Đông Nam Bộ.|Tây Nguyên.,Duyên hải miền Trung.,Bắc Trung Bộ.,1.0,"bão , lũ lụt , hạn hán , gió tây khô nóng là thiên tai xảy ra chủ yếu ở vùng ngoại ô | đồng bằng sông hồng .| tây bắc .| duyên hải miền trung .| tây nguyên",vùng nào ở nước ta chịu ảnh hưởng mạnh mẽ nhất của gió mạnh tây khô nóng ?| bắc trung bộ .| đông bắc .| đông nam bộ .| tây nguyên .
...,...,...,...,...,...,...,...
8622,"Việc phát triển kinh tế - xã hội vùng dân tộc ít người ở nước ta cần được chú trọng hơn nữa do|các dân tộc ít người đóng vai trò rất quan trọng trong việc đảm bảo an ninh quốc phòng.|một số dân tộc ít người có những kinh nghiệm sản xuất quí báu.|sự phát triển kinh tế - xã hội giữa các dân tộc hiện có sự chênh lệch đáng kể, mức sống của bộ phận dân tộc ít người thấp.|trước đây chúng ta chưa chú trọng vấn đề này.","Đặc điểm nào không đúng với dân cư, dân tộc ở nước t

In [12]:
augmented_df =  augmented_df[['augmented1', 'augmented2', 'ans_question_1', 'ans_question_2', 'manual_label']]
augmented_df['cosine_similarity'] = ''
augmented_df  = augmented_df[['augmented1', 'augmented2', 'ans_question_1', 'ans_question_2', 'cosine_similarity','manual_label']]
augmented_df = augmented_df.set_axis(['question_1', 'question_2', 'ans_question_1', 'ans_question_2', 'cosine_similarity','manual_label'], axis=1, inplace=False)
augmented_df.head(2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,question_1,question_2,ans_question_1,ans_question_2,cosine_similarity,manual_label
0,"“ dân cư và lao động ” | dân cư đông , nguồn lao động dồi dào và có trình độ .| nguồn lao động lớn nhất cả nước .| lao động có trình độ cao nhất cả nước .| lao động tập trung chủ yếu ở các thành phố lớn .","tại sao việc làm là một trong những vấn đề nan giải ở đồng bằng sông hồng nhất là ở khu vực thành thị ?| do dân nhập cư đông .| do dân cư đông , kết cấu dân cư trẻ .| do nền kinh tế còn chậm phát triển .| do dân cư đông , kết cấu dân cư trẻ trong điều kiện kinh tế chậm","dân số đông, nguồn lao động dồi dào và có trình độ.","Do dân số đông, kết cấu dân số trẻ trong điều kiện kinh tế chậm",,1.0
1,"bão , lũ lụt , hạn hán , gió mạnh tây khô nóng là thiên tai xảy ra chủ yếu ở vùng | đồng bằng sông hồng .| tây bắc .| duyên hải miền trung .| tây nguyên",vùng nào ở nước ta chịu ảnh hưởng mạnh mẽ nhất của gió mạnh tây khô nóng ?| bắc trung bộ .| đông bắc .| đông nam bộ .| tây nguyên .,Duyên hải miền Trung.,Bắc Trung Bộ.,,1.0


In [13]:
merged_df = original_df
merged_df = merged_df.append(augmented_sem0, sort=False, ignore_index=True)
merged_df = merged_df.append(augmented_df, sort=False, ignore_index=True)

In [14]:
merged_df.tail(2)

,question_1,question_2,ans_question_1,ans_question_2,cosine_similarity,manual_label
13234,"việc phát ra kinh tế - xã hội vùng dân tộc ít người ở nước ta cần được chú trọng hơn nữa do | các dân tộc ít người đóng vai trò rất quan trọng trong việc đảm bảo an ninh quốc phòng .| một số dân tộc ít người có những kinh nghiệm sản xuất quí báu .| sự phát ra kinh tế - xã hội giữa các dân tộc hiện có sự chênh lệch đáng kể , mức sống của bộ phận dân tộc ít người thấp .| trước đây chúng ta chưa chú trọng vấn đề này .","đặc điểm nào không đúng với dân cư , dân tộc ở nước ta ?| các dân tộc luôn phát huy truyền thống sản xuất .| các dân tộc luôn đoàn kết bên nhau .| chất lượng đời sống của các dân tộc ít người đã ở mức cao .| sự phát triển kinh tế - xã hội giữa các vùng còn chênh lệch .","sự phát triển kinh tế - xã hội giữa các dân tộc hiện có sự chênh lệch đáng kể, mức sống của bộ phận dân tộc ít người thấp.",Chất lượng đời sống của các dân tộc ít người đã ở mức cao.,,1.0
13235,"việc phát ra kinh tế - xã hội vùng dân tộc ít người ở nước ta cần được chú trọng hơn nữa do | các dân tộc ít người đóng vai trò rất quan trọng trong việc đảm bảo an ninh quốc phòng .| một số dân tộc ít người có những kinh nghiệm sản xuất quí báu .| sự phát ra kinh tế - xã hội giữa các dân tộc hiện có sự chênh lệch đáng kể , mức sống của bộ phận dân tộc ít người thấp .| trước đây chúng ta chưa chú trọng vấn đề này .","đặc điểm nào không đúng với dân cư , dân tộc ở nước ta ?| các dân tộc luôn phát huy truyền thống sản xuất .| các dân tộc luôn đoàn kết bên nhau .| chất lượng đời sống của các dân tộc ít người đã ở mức cao .| sự phát ra kinh tế - xã hội giữa các vùng còn chênh lệch .","sự phát triển kinh tế - xã hội giữa các dân tộc hiện có sự chênh lệch đáng kể, mức sống của bộ phận dân tộc ít người thấp.",Chất lượng đời sống của các dân tộc ít người đã ở mức cao.,,1.0


In [17]:
merged_df = merged_df.sample(frac = 1)
merged_df = merged_df.reset_index(drop=True)
merged_df

,question_1,question_2,ans_question_1,ans_question_2,cosine_similarity,manual_label
0,"căn cứ vào atlat địa lí trang 15 , cho phép biết cơ cấu lao động theo các ngành kinh tế của nước ta đang có sự chuyển dịch theo hướng nào dưới đây ?| tăng tỉ trọng lao động ở khu vực ngoài nhà nước .| tăng tỉ trọng lao động ở khu vực có vốn đầu tư nước ngoài .| giảm tỉ trọng lao động ở khu vực nông - lâm - ngư nghiệp .| giảm tỉ trọng lao động ở khu vực công nghiệp - xây dựng .",nguồn lực lượng lao động nước ta dồi dào là điều kiện thuận lợi : | phát triển những ngành đòi hỏi nhiều lực lượng lao động .| đẩy mạnh đào tạo nghề cho người lực lượng lao động .| tăng thêm lực lượng lực lượng lao động có chuyên môn kĩ thuật .| dễ dàng tiếp thu khoa học kĩ thuật tiên tiến .,giảm tỉ trọng lao động ở khu vực nông - lâm - ngư nghiệp.,Phát triển những ngành đòi hỏi nhiều lao động.,,1.0
1,"đâu là đặc điểm đúng với hoạt động của bão ở việt nam ?| mùa hè bão bắt đầu từ tháng iv và kết thúc vào tháng xi .| mùa hè bão chậm dần từ nam ra bắc .| 70% số cơn bão trong mùa hè tập trung vào các tháng viii , ix , x .| trung bình mỗi năm có 10 đến 12 cơn bão đổ bộ vào vùng biển nước ta .",nguyên nhân chủ yếu làm cho mùa bão nước ta chậm chạp dần từ bắc vào nam là | hình dạng lãnh thổ hẹp ngang và kéo dài theo chiều bắc - nam .| gió mùa đông bắc suy dần khi di chuyển xuống phía nam .| dải hội tụ nhiệt đới lùi dần từ bắc vào nam và hoạt động của bão .| nước ta tiếp giáp với biển đông rộng lớn .,"70% số cơn bão trong mùa tập trung vào các tháng VIII, IX, X.",dải hội tụ nhiệt đới lùi dần từ bắc vào nam và hoạt động của bão.,,1.0
2,"cần phải đặt vấn đề sử dụng hợp lý và bảo vệ tài nguyên thiên nhiên nước ta , vì : | thiên nhiên , bão lũ , hạn hán thường xuyên xảy ra .| đảm bảo tốt nhất cho mượn sự phát triển bền vững lâu dài .| dân số tăng nhanh , đời sống xã hội nâng cao .| khoa học kỹ thuật có nhiều tiến bộ .",để bảo hiểm tốt nguồn tài nguyên cho sự phát triển bền vững lâu dài thì | cần đặt ra vấn đề sử dụng hợp lý và bảo vệ tài nguyên thiên nhiên .| quản lí các chất thải độc hại từ các nhà máy xả thải vào môi trường .| bảo vệ và quản lí chặt chẽ các nguồn tài nguyên thiên nhiên .| quản lí chặt chẽ việc khai thác và sử dụng tài nguyên thiên nhiên .,đảm bảo tốt nhất cho sự phát triển bền vững lâu dài.,cần đặt ra vấn đề sử dụng hợp lý và bảo vệ tài nguyên thiên nhiên.,,1.0
3,"cơ cấu lao động theo thành thị và nông thôn không có sự biến đổi theo hướng nào dưới đây ?.| tỉ trọng lao động ở thành thị tăng .| tỉ trọng lao động ở nông thôn giảm .| tỉ trọng nông thôn giảm , thành thị tăng .| tỉ trọng nông thôn tăng , thành thị giảm .","cơ cấu lao động theo thành phần kinh tế ở nước ta có sự sự thay đổi thế nào ?| tỉ trọng lao động trong nhà nước giảm .| tỉ trọng lao động ngoài nhà nước tăng .| tỉ trọng lao động trong khu vực có vốn đầu tư nước ngoài tăng nhanh .| tỉ trọng lao động trong nhà nước , ngoài nhà nước giảm .","tỉ trọng nông thôn tăng, thành thị giảm.",Tỉ trọng lao động trong khu vực có vốn đầu tư nước ngoài tăng nhanh.,,1.0
4,"căn cứ vào atlat địa lí việt nam trang 9 , hãy cho biết địa điểm nào sau đây có đo lường mưa trung bình năm trên 2800 mm / năm ?| lạng sơn .| hà nội .| thừa thiên – huế .| tp .hồ chí minh .",địa điểm nào dưới đây có lượng lượng mưa trung bình năm lớn nhât ?| hà nội .| huế .| nha trang .| phan thiết .,Thừa Thiên – Huế.,Huế.,,1.0
...,...,...,...,...,...,...
13231,"căn cứ vào atlat địa lí việt nam trang 9 , hãy cho phép biết ở nước ta tần suất hoạt động của bão nhiều nhất vào tháng nào trong năm ?| tháng 10 .| tháng 11 .| tháng 9 .| tháng 7 .",nguyên nhân chủ yếu làm cho phép mùa bão nước ta chậm dần từ bắc vào nam là | hình dạng lãnh thổ hẹp ngang và kéo dài theo chiều bắc - nam .| gió mùa đông bắc suy dần khi di chuyển xuống phía nam .| dải hội tụ nhiệt đới lùi dần từ bắc vào nam và hoạt động của bão .| nước ta tiếp giáp với biển đông rộng lớn .,Tháng 9.,dải hội tụ nhiệt đới lùi dần từ bắc vào nam và hoạt động của bão.,,1.0
13232,"căn cứ vào atla

In [18]:
dataset = merged_df.set_axis(['sentence1', 'sentence2', 'ans_sentence1', 'ans_sentence2', 'similarity', 'label'], axis=1, inplace=False)

In [19]:
dataset

,sentence1,sentence2,ans_sentence1,ans_sentence2,similarity,label
0,"căn cứ vào atlat địa lí trang 15 , cho phép biết cơ cấu lao động theo các ngành kinh tế của nước ta đang có sự chuyển dịch theo hướng nào dưới đây ?| tăng tỉ trọng lao động ở khu vực ngoài nhà nước .| tăng tỉ trọng lao động ở khu vực có vốn đầu tư nước ngoài .| giảm tỉ trọng lao động ở khu vực nông - lâm - ngư nghiệp .| giảm tỉ trọng lao động ở khu vực công nghiệp - xây dựng .",nguồn lực lượng lao động nước ta dồi dào là điều kiện thuận lợi : | phát triển những ngành đòi hỏi nhiều lực lượng lao động .| đẩy mạnh đào tạo nghề cho người lực lượng lao động .| tăng thêm lực lượng lực lượng lao động có chuyên môn kĩ thuật .| dễ dàng tiếp thu khoa học kĩ thuật tiên tiến .,giảm tỉ trọng lao động ở khu vực nông - lâm - ngư nghiệp.,Phát triển những ngành đòi hỏi nhiều lao động.,,1.0
1,"đâu là đặc điểm đúng với hoạt động của bão ở việt nam ?| mùa hè bão bắt đầu từ tháng iv và kết thúc vào tháng xi .| mùa hè bão chậm dần từ nam ra bắc .| 70% số cơn bão trong mùa hè tập trung vào các tháng viii , ix , x .| trung bình mỗi năm có 10 đến 12 cơn bão đổ bộ vào vùng biển nước ta .",nguyên nhân chủ yếu làm cho mùa bão nước ta chậm chạp dần từ bắc vào nam là | hình dạng lãnh thổ hẹp ngang và kéo dài theo chiều bắc - nam .| gió mùa đông bắc suy dần khi di chuyển xuống phía nam .| dải hội tụ nhiệt đới lùi dần từ bắc vào nam và hoạt động của bão .| nước ta tiếp giáp với biển đông rộng lớn .,"70% số cơn bão trong mùa tập trung vào các tháng VIII, IX, X.",dải hội tụ nhiệt đới lùi dần từ bắc vào nam và hoạt động của bão.,,1.0
2,"cần phải đặt vấn đề sử dụng hợp lý và bảo vệ tài nguyên thiên nhiên nước ta , vì : | thiên nhiên , bão lũ , hạn hán thường xuyên xảy ra .| đảm bảo tốt nhất cho mượn sự phát triển bền vững lâu dài .| dân số tăng nhanh , đời sống xã hội nâng cao .| khoa học kỹ thuật có nhiều tiến bộ .",để bảo hiểm tốt nguồn tài nguyên cho sự phát triển bền vững lâu dài thì | cần đặt ra vấn đề sử dụng hợp lý và bảo vệ tài nguyên thiên nhiên .| quản lí các chất thải độc hại từ các nhà máy xả thải vào môi trường .| bảo vệ và quản lí chặt chẽ các nguồn tài nguyên thiên nhiên .| quản lí chặt chẽ việc khai thác và sử dụng tài nguyên thiên nhiên .,đảm bảo tốt nhất cho sự phát triển bền vững lâu dài.,cần đặt ra vấn đề sử dụng hợp lý và bảo vệ tài nguyên thiên nhiên.,,1.0
3,"cơ cấu lao động theo thành thị và nông thôn không có sự biến đổi theo hướng nào dưới đây ?.| tỉ trọng lao động ở thành thị tăng .| tỉ trọng lao động ở nông thôn giảm .| tỉ trọng nông thôn giảm , thành thị tăng .| tỉ trọng nông thôn tăng , thành thị giảm .","cơ cấu lao động theo thành phần kinh tế ở nước ta có sự sự thay đổi thế nào ?| tỉ trọng lao động trong nhà nước giảm .| tỉ trọng lao động ngoài nhà nước tăng .| tỉ trọng lao động trong khu vực có vốn đầu tư nước ngoài tăng nhanh .| tỉ trọng lao động trong nhà nước , ngoài nhà nước giảm .","tỉ trọng nông thôn tăng, thành thị giảm.",Tỉ trọng lao động trong khu vực có vốn đầu tư nước ngoài tăng nhanh.,,1.0
4,"căn cứ vào atlat địa lí việt nam trang 9 , hãy cho biết địa điểm nào sau đây có đo lường mưa trung bình năm trên 2800 mm / năm ?| lạng sơn .| hà nội .| thừa thiên – huế .| tp .hồ chí minh .",địa điểm nào dưới đây có lượng lượng mưa trung bình năm lớn nhât ?| hà nội .| huế .| nha trang .| phan thiết .,Thừa Thiên – Huế.,Huế.,,1.0
...,...,...,...,...,...,...
13231,"căn cứ vào atlat địa lí việt nam trang 9 , hãy cho phép biết ở nước ta tần suất hoạt động của bão nhiều nhất vào tháng nào trong năm ?| tháng 10 .| tháng 11 .| tháng 9 .| tháng 7 .",nguyên nhân chủ yếu làm cho phép mùa bão nước ta chậm dần từ bắc vào nam là | hình dạng lãnh thổ hẹp ngang và kéo dài theo chiều bắc - nam .| gió mùa đông bắc suy dần khi di chuyển xuống phía nam .| dải hội tụ nhiệt đới lùi dần từ bắc vào nam và hoạt động của bão .| nước ta tiếp giáp với biển đông rộng lớn .,Tháng 9.,dải hội tụ nhiệt đới lùi dần từ bắc vào nam và hoạt động của bão.,,1.0
13232,"căn cứ vào atlat địa lí việt nam 

**Train Test Split**

In [20]:
!pip install sklearn
from sklearn.model_selection import train_test_split

In [21]:
df_train, df_test = train_test_split(dataset, test_size=0.2, random_state = 8, shuffle=True)
df_train, df_val = train_test_split(dataset, test_size=0.25, random_state = 8)

In [22]:
print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

(9927, 6)
(3309, 6)
(2648, 6)


In [23]:
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)

In [24]:
df_train

,sentence1,sentence2,ans_sentence1,ans_sentence2,similarity,label
0,"thiên tai xảy ra hằng năm , đe doạ và gây hậu quả nặng nề nhất cho vùng ngoại ô đồng bằng , ven biển nước ta là : | bão .| sạt lở bờ biển .| cát bay , cát chảy .| động đất .","bão là thiên tai xảy ra hằng năm , đe doạ và gây hậu quả nặng nề nhất cho vùng ngoại ô nào ở nước ta hiện nay ?| vùng ngoại ô đồng bằng , ven biển .| vùng ngoại ô đồi núi , ven biển .| vùng ngoại ô trung du , đồng bằng .| vùng ngoại ô trung du và miền núi .",Bão.,"Vùng đồng bằng, ven biển.",,1.0
1,"cơ cấu lao động theo thành phần kinh tế ở nước ta có sự thay đổi thế nào ?| tỉ trọng lao động trong nhà nước giảm .| tỉ trọng lao động ngoài nhà nước tăng .| tỉ trọng lao động trong khu vực có vốn đầu tư nước ngoài tăng nhanh .| tỉ trọng lao động trong nhà nước , ngoài nhà nước giảm .","sự thay đổi cơ cấu lực lượng lao động theo các ngành kinh tế của nước ta hiện nay chủ yếu do tác động của | việc thực hiện kế hoạch hoá gia đình .| sản xuất nông – lâm – ngư nghiệp cần nhiều lực lượng lao động .| chuyển dịch cơ cấu kinh tế theo công nghiệp hoá , hiện đại hoá .| phân bố lại dân cư – lực lượng lao động giữa các vùng .",Tỉ trọng lao động trong khu vực có vốn đầu tư nước ngoài tăng nhanh.,"chuyển dịch cơ cấu kinh tế theo công nghiệp hóa, hiện đại hóa.",,1.0
2,"đàn gia cầm ở nước ta có xu hướng không ngừng tăng lên chủ yếu là do | có nguồn thức ăn dồi dào từ ngành trồng trọt .| nhu cầu thịt , trứng cho tiêu dùng ngày càng tăng .| dịch vụ thú y được rồi chú trọng phát triển .| chính sách phát triển chăn nuôi của nhà nước .","khó khăn lớn nhất trong việc xuất khẩu các sản phẩm chăn nuôi của nước ta trong giai đoạn hiện nay là | giá trị dinh dưỡng của sản phẩm thấp .| nguồn thức ăn cho chăn nuôi chưa được rồi đảm bảo .| công tác kiểm dịch , vệ sinh an toàn thực phẩm chưa được rồi chú ý đúng mức .| giá thành sản phẩm còn cao .","nhu cầu thịt, trứng cho tiêu dùng ngày càng tăng.","công tác kiểm dịch, vệ sinh an toàn thực phẩm chưa được chú ý đúng mức.",,1.0
3,"cơ cấu lao động theo thành thị và nông thôn không có sự thay đổi theo hướng nào dưới điều đó ?.| tỉ trọng lao động ở thành thị tăng .| tỉ trọng lao động ở nông thôn giảm .| tỉ trọng nông thôn giảm , thành thị tăng .| tỉ trọng nông thôn tăng , thành thị giảm .",hướng giải quyết việc làm cho người lực lượng lao động nước ta nào sau đây chủ yếu tập trung vào người lực lượng lao động ?| thực hiện đa dạng hoá các hoạt động sản xuất .| phân bố lại dân cư và nguồn lực lượng lao động .| tăng cường hợp tác liên kết để thu hút vốn đầu tư nước ngoài .| mở rộng sản xuất hàng xuất khẩu .,"tỉ trọng nông thôn tăng, thành thị giảm.",Phân bố lại dân cư và nguồn lao động.,,1.0
4,"nguyên nhân quan trọng nhất về mặt tự nhiên để cây cao su phát triển mạnh ở đông nam bộ là : | nguồn nước mặt phong phú .| có nhiều cơ sở chế biến mủ cao su trong vùng ngoại ô .| thị trường tiêu thụ cao su rộng lớn và ổn định .| có loại đất xám rất thích hợp cho cây cao su , khí hậu nóng ẩm , ít bão .","vấn đề phát ra tổng hợp kinh tế biển có ảnh hưởng rất lớn đến sự hình thành và thay đổi cơ cấu lãnh thổ kinh tế của đông nam bộ , vì kinh tế biển | đa dạng về ngành .| gắn liền với vùng ven biển .| mang lại hiệu quả cao .| tác động đến nhiều khu vực kinh tế khác .","có loại đất xám rất thích hợp cho cây cao su, khí hậu nóng ẩm, ít bão.",mang lại hiệu quả cao.,,1.0
...,...,...,...,...,...,...
9922,"khó khăn lớn nhất về mặt tự nhiên đối với việc phát triển kinh tế - xã hội của vùng đồi núi là | đất trồng cây lương thực bị hạn chế .| địa hình bị chia cắt mạnh , nhiều sông suối , hẻm vực trở ngại cho giao thông .| khí hậu phân hoá phức tạp .| khoáng sản có nhiều mỏ trữ lượng nhỏ , phân tán trong không gian .","vì sao việc khai thác , sử dụng hợp lí miền đồi núi có ảnh hưởng lớn tới sự phát triển kinh tế - xã hội , bảo vệ sinh thái cho vùng ngoại ô đồng bằng ?| miền núi nước ta giàu tài nguyên khoáng sản có nguồn gốc nội sinh .| địa hình nước ta chủ yếu là đồi núi thấp , có sự phân hoá đa dạng .

# Load Bert Model

In [28]:
from transformers import BertTokenizer, BertModel

Tokenize for our dataset using BertTokenizer

In [29]:
class CustomDataset(Dataset):

    def __init__(self, data, maxlen, with_labels=True, bert_model='bert-base-cased'):

        self.data = data  # pandas dataframe
        #Initialize the tokenizer
        self.tokenizer = BertTokenizer.from_pretrained(bert_model)  

        self.maxlen = maxlen
        self.with_labels = with_labels 

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):

        # Selecting sentence1 and sentence2 at the specified index in the data frame
        sent1 = str(self.data.loc[index, 'sentence1'])
        sent2 = str(self.data.loc[index, 'sentence2'])

        # Tokenize the pair of sentences to get token ids, attention masks and token type ids
        encoded_pair = self.tokenizer(sent1, sent2, 
                                      padding='max_length',  # Pad to max_length
                                      truncation=True,  # Truncate to max_length
                                      max_length=self.maxlen,  
                                      return_tensors='pt')  # Return torch.Tensor objects
        
        token_ids = encoded_pair['input_ids'].squeeze(0)  # tensor of token ids
        attn_masks = encoded_pair['attention_mask'].squeeze(0)  # binary tensor with "0" for padded values and "1" for the other values
        token_type_ids = encoded_pair['token_type_ids'].squeeze(0)  # binary tensor with "0" for the 1st sentence tokens & "1" for the 2nd sentence tokens

        if self.with_labels:  # True if the dataset has labels
            label = self.data.loc[index, 'label']
            return token_ids, attn_masks, token_type_ids, label  
        else:
            return token_ids, attn_masks, token_type_ids

In [30]:
class SentencePairClassifier(nn.Module):

    def __init__(self, bert_model="bert-base-cased", freeze_bert=False):
        super(SentencePairClassifier, self).__init__()
        #  Instantiating BERT-based model object
        self.bert_layer = BertModel.from_pretrained(bert_model, return_dict=False)
        hidden_size = 768

        # Freeze bert layers and only train the classification layer weights
        if freeze_bert:
            for p in self.bert_layer.parameters():
                p.requires_grad = False

        # Classification layer
        self.cls_layer = nn.Linear(hidden_size, 1)

        self.dropout = nn.Dropout(p=0.1)

    @autocast()  # run in mixed precision
    def forward(self, input_ids, attn_masks, token_type_ids):
        '''
        Inputs:
            -input_ids : Tensor  containing token ids
            -attn_masks : Tensor containing attention masks to be used to focus on non-padded values
            -token_type_ids : Tensor containing token type ids to be used to identify sentence1 and sentence2
        '''

        # Feeding the inputs to the BERT-based model to obtain contextualized representations
        # cont_reps, pooler_output = self.bert_layer(input_ids, attn_masks, token_type_ids)
        cont_reps, pooler_output = self.bert_layer(input_ids, attn_masks, token_type_ids)

        # Feeding to the classifier layer the last layer hidden-state of the [CLS] token further processed by a
        # Linear Layer and a Tanh activation. The Linear layer weights were trained from the sentence order prediction (ALBERT) or next sentence prediction (BERT)
        # objective during pre-training.
        logits = self.cls_layer(self.dropout(pooler_output))
        
        return logits

In [31]:
def set_seed(seed):
    """ Set all seeds to make results reproducible """
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    

def evaluate_loss(net, device, criterion, dataloader):
    net.eval()

    mean_loss = 0
    count = 0

    with torch.no_grad():
        for it, (seq, attn_masks, token_type_ids, labels) in enumerate(tqdm(dataloader)):
            seq, attn_masks, token_type_ids, labels = \
                seq.to(device), attn_masks.to(device), token_type_ids.to(device), labels.to(device)
            logits = net(seq, attn_masks, token_type_ids)
            mean_loss += criterion(logits.squeeze(-1), labels.float()).item()
            count += 1

    return mean_loss / count

In [32]:
print("Creation of the models' folder...")
!mkdir models

Creation of the models' folder...


In [33]:
def train_bert(net, criterion, opti, lr, lr_scheduler, train_loader, val_loader, epochs, iters_to_accumulate):

    best_loss = np.Inf
    best_ep = 1
    nb_iterations = len(train_loader)
    print_every = nb_iterations // 5  # print the training loss 5 times per epoch
    iters = []
    train_losses = []
    val_losses = []

    scaler = GradScaler()

    for ep in range(epochs):

        net.train()
        running_loss = 0.0
        for it, (seq, attn_masks, token_type_ids, labels) in enumerate(tqdm(train_loader)):

            # Converting to cuda tensors
            seq, attn_masks, token_type_ids, labels = \
                seq.to(device), attn_masks.to(device), token_type_ids.to(device), labels.to(device)
    
            # Enables autocasting for the forward pass (model + loss)
            with autocast():
                # Obtaining the logits from the model
                logits = net(seq, attn_masks, token_type_ids)

                # Computing loss
                loss = criterion(logits.squeeze(-1), labels.float())
                loss = loss / iters_to_accumulate  # Normalize the loss because it is averaged

            # Backpropagating the gradients
            # Scales loss.  Calls backward() on scaled loss to create scaled gradients.
            scaler.scale(loss).backward()

            if (it + 1) % iters_to_accumulate == 0:
                # Optimization step
                # scaler.step() first unscales the gradients of the optimizer's assigned params.
                # If these gradients do not contain infs or NaNs, opti.step() is then called,
                # otherwise, opti.step() is skipped.
                scaler.step(opti)
                # Updates the scale for next iteration.
                scaler.update()
                # Adjust the learning rate based on the number of iterations.
                lr_scheduler.step()
                # Clear gradients
                opti.zero_grad()


            running_loss += loss.item()

            if (it + 1) % print_every == 0:  # Print training loss information
                print()
                print("Iteration {}/{} of epoch {} complete. Loss : {} "
                      .format(it+1, nb_iterations, ep+1, running_loss / print_every))

                running_loss = 0.0


        val_loss = evaluate_loss(net, device, criterion, val_loader)  # Compute validation loss
        print()
        print("Epoch {} complete! Validation Loss : {}".format(ep+1, val_loss))

        if val_loss < best_loss:
            print("Best validation loss improved from {} to {}".format(best_loss, val_loss))
            print()
            net_copy = copy.deepcopy(net)  # save a copy of the model
            best_loss = val_loss
            best_ep = ep + 1

    # Saving the model
    path_to_model='models/{}_lr_{}_val_loss_{}_ep_{}.pt'.format(bert_model, lr, round(best_loss, 5), best_ep)
    torch.save(net_copy.state_dict(), path_to_model)
    print("The model has been saved in {}".format(path_to_model))

    del loss
    torch.cuda.empty_cache()

In [34]:
bert_model = "bert-base-cased"
freeze_bert = False  # if True, freeze the encoder weights and only update the classification layer weights
maxlen = 256
bs = 8
iters_to_accumulate = 2  # the gradient accumulation adds gradients over an effective batch of size : bs * iters_to_accumulate. If set to "1", you get the usual batch size
lr = 2e-5
epochs = 4

# Train and Evaluation

In [35]:
#  Set all seeds to make reproducible results
set_seed(1)

# Creating instances of training and validation set
print("Reading training data...")
train_set = CustomDataset(df_train, maxlen, bert_model)
print("Reading validation data...")
val_set = CustomDataset(df_val, maxlen, bert_model)
# Creating instances of training and validation dataloaders
train_loader = DataLoader(train_set, batch_size=bs, num_workers=5,shuffle=True)
val_loader = DataLoader(val_set, batch_size=bs, num_workers=5,shuffle=True)


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = SentencePairClassifier(bert_model, freeze_bert=freeze_bert)

if torch.cuda.device_count() > 1:  # if multiple GPUs
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    net = nn.DataParallel(net)

net.to(device)

criterion = nn.BCEWithLogitsLoss()
opti = AdamW(net.parameters(), lr=lr, weight_decay=1e-2)
num_warmup_steps = 0 # The number of steps for the warmup phase.
num_training_steps = epochs * len(train_loader)  # The total number of training steps
t_total = (len(train_loader) // iters_to_accumulate) * epochs  # Necessary to take into account Gradient accumulation
lr_scheduler = get_linear_schedule_with_warmup(optimizer=opti, num_warmup_steps=num_warmup_steps, num_training_steps=t_total)

train_bert(net, criterion, opti, lr, lr_scheduler, train_loader, val_loader, epochs, iters_to_accumulate)

Reading training data...


Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Reading validation data...


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

...


Iteration 992/1241 of epoch 4 complete. Loss : 0.04145645784140675 


100%|██████████| 1241/1241 [03:06<00:00,  7.09it/s]


Iteration 1240/1241 of epoch 4 complete. Loss : 0.03953507181816554 


100%|██████████| 414/414 [00:20<00:00, 19.73it/s]



Epoch 4 complete! Validation Loss : 0.0861950509502116
Best validation loss improved from 0.08820384346704552 to 0.0861950509502116

The model has been saved in models/bert-base-cased_lr_2e-05_val_loss_0.0862_ep_4.pt


# Predictions

In [36]:
def get_probs_from_logits(logits):
    """
    Converts a tensor of logits into an array of probabilities by applying the sigmoid function
    """
    probs = torch.sigmoid(logits.unsqueeze(-1))
    return probs.detach().cpu().numpy()

def test_prediction(net, device, dataloader, with_labels=True, result_file="results/output.txt"):
    """
    Predict the probabilities on a dataset with or without labels and print the result in a file
    """
    net.eval()
    w = open(result_file, 'w')
    probs_all = []

    with torch.no_grad():
        if with_labels:
            for seq, attn_masks, token_type_ids, _ in tqdm(dataloader):
                seq, attn_masks, token_type_ids = seq.to(device), attn_masks.to(device), token_type_ids.to(device)
                logits = net(seq, attn_masks, token_type_ids)
                probs = get_probs_from_logits(logits.squeeze(-1)).squeeze(-1)
                probs_all += probs.tolist()
        else:
            for seq, attn_masks, token_type_ids in tqdm(dataloader):
                seq, attn_masks, token_type_ids = seq.to(device), attn_masks.to(device), token_type_ids.to(device)
                logits = net(seq, attn_masks, token_type_ids)
                probs = get_probs_from_logits(logits.squeeze(-1)).squeeze(-1)
                probs_all += probs.tolist()

    w.writelines(str(prob)+'\n' for prob in probs_all)
    w.close()

In [37]:
path_to_model = '/content/models/bert-base-cased_lr_2e-05_val_loss_0.0862_ep_4.pt'  

path_to_output_file = 'results/output.txt'

print("Reading test data...")
test_set = CustomDataset(df_test, maxlen, bert_model)
test_loader = DataLoader(test_set, batch_size=bs, num_workers=5)

model = SentencePairClassifier(bert_model)
if torch.cuda.device_count() > 1:  # if multiple GPUs
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)

print()
print("Loading the weights of the model...")
model.load_state_dict(torch.load(path_to_model))
model.to(device)

print("Predicting on test data...")
test_prediction(net=model, device=device, dataloader=test_loader, with_labels=True,  # set the with_labels parameter to False if your want to get predictions on a dataset without labels
                result_file=path_to_output_file)
print()
print("Predictions are available in : {}".format(path_to_output_file))

Reading test data...


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



Loading the weights of the model...
Predicting on test data...


100%|██████████| 331/331 [00:15<00:00, 20.81it/s]


Predictions are available in : results/output.txt


# Evaluation

In [38]:
path_to_output_file = 'results/output.txt'  # path to the file with prediction probabilities

labels_test = df_test['label']  # true labels

probs_test = pd.read_csv(path_to_output_file, header=None)[0]  # prediction probabilities
threshold = 0.5   # you can adjust this threshold for your own dataset
preds_test=(probs_test>=threshold).astype('uint8') # predicted labels using the above fixed threshold

metric = load_metric("glue", "mrpc")

https://raw.githubusercontent.com/huggingface/datasets/1.0.1/metrics/glue/glue.py not found in cache or force_download set to True, downloading to /root/.cache/huggingface/datasets/tmpxq6_p12v


Downloading:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

storing https://raw.githubusercontent.com/huggingface/datasets/1.0.1/metrics/glue/glue.py in cache at /root/.cache/huggingface/datasets/50d5843bbbbd80c47809bc76a5b03c0fd87d068509b0060103ae8182e4f5cfb9.ec871b06a00118091ec63eff0a641fddcb8d3c7cd52e855bbb2be28944df4b82.py
creating metadata file for /root/.cache/huggingface/datasets/50d5843bbbbd80c47809bc76a5b03c0fd87d068509b0060103ae8182e4f5cfb9.ec871b06a00118091ec63eff0a641fddcb8d3c7cd52e855bbb2be28944df4b82.py
Checking /root/.cache/huggingface/datasets/50d5843bbbbd80c47809bc76a5b03c0fd87d068509b0060103ae8182e4f5cfb9.ec871b06a00118091ec63eff0a641fddcb8d3c7cd52e855bbb2be28944df4b82.py for additional imports.
Creating main folder for metric https://raw.githubusercontent.com/huggingface/datasets/1.0.1/metrics/glue/glue.py at /root/.cache/huggingface/modules/datasets_modules/metrics/glue
Creating specific version folder for metric https://raw.githubusercontent.com/huggingface/datasets/1.0.1/metrics/glue/glue.py at /root/.cache/huggingface/mod

In [39]:
# Compute the accuracy and F1 scores
metric._compute(predictions=preds_test, references=labels_test)

{'accuracy': 0.9543051359516617, 'f1': 0.967199783139062}

In [40]:
df_test.loc[15]

sentence1        Kim ngạch xuất nhập khẩu của đất nước chúng tôi liên tục tăng chủ yếu do thị trường thế giới ngày càng mở rộng.|.Đa dạng hóa các đối tượng tham gia vào các hoạt động xuất nhập khẩu.|.Tăng cường nhập khẩu các dòng máy móc, toàn bộ thiết bị và hàng tiêu dùng.|.Sự phát triển của nền kinh tế trong nước và đổi mới trong các cơ chế quản lý.
sentence2                                                                                                                                                          Trên cơ sở địa lý của Việt Nam Atlat, các lĩnh vực khí đốt được vận hành ở nước ta (2007): |Đỏ, Lan Tây, Rồng.|.Đỏ, Lan Tây, Tiên Hải.|.Bạc Hải, Lan đỏ, Đại Hùng.|.Hồng Ngọc, Rồng, Tiên Hải.
ans_sentence1                                                                                                                                                                                                                                                                                       